In [1]:
import random
import numpy as np
import json


def optimize(function, dimensions, lower_boundary, upper_boundary, max_iter, maximize=False):
    best_solution = np.array([float()] * dimensions)

    for i in range(dimensions):
        best_solution[i] = random.uniform(lower_boundary[i], upper_boundary[i])

    for _ in range(max_iter):

        solution1 = function(best_solution)

        new_solution = [lower_boundary[d] + random.random() * (upper_boundary[d] - lower_boundary[d]) for d in
                        range(dimensions)]

        if np.greater_equal(new_solution, lower_boundary).all() and np.less_equal(new_solution, upper_boundary).all():
            solution2 = function(new_solution)
        elif maximize:
            solution2 = -100000.0
        else:
            solution2 = 100000.0

        if solution2 > solution1 and maximize:
            best_solution = new_solution
        elif solution2 < solution1 and not maximize:
            best_solution = new_solution

    best_fitness = function(best_solution)

    return best_fitness, best_solution
def ClearoldExperimentDATA(expname):
    CurrentPath = pathlib.Path().parent.resolve()
    try: 
        oldpath = r"{}/{}".format(CurrentPath,expname)
        print("Deleting {}".format(oldpath))
        shutil.rmtree(oldpath)
    except:
        print("No Files found, Skipping delete")


def MakeSampleRandom(trackid=1, precision=2,):

        
    N=precision
    x = {
        "param": [
            {
                "track": trackidSTR,
                "horse_max_age": random.randint(10,40), 
                "horse_min_hit":random.randint(5,40),
                "full_max_age": random.randint(10,40),
                "full_min_hit": random.randint(10,40),
                "fstall_normal_conf": round(random.uniform(0.5,0.9), N),
                "fstall_corner_conf": round(random.uniform(0.4,0.8), N),
                "bstall_normal_conf": round(random.uniform(0.4,0.8), N),
                "bstall_corner_conf": round(random.uniform(0.4,0.9), N),
                "counter_regulator": round(random.uniform(0.4,0.8), N),
                "activate_flage": 1,
                "flow_thresh": 30,
                "about_thresh":2500,
                "horse_fraction":round(random.uniform(0.10,0.25), 2),
                "Fn_delay":[91,random.randint(3, 9)],
                "Ld_delay":[91,random.randint(10, 50),
                            random.randint(4, 25)],
                "activate_bl": 1,
                "objects_confidence": [
                    round(random.uniform(0.25,0.50), 2),
                    0.2,
                    0.2,
                    0.3,
                    0.1,
                    0.3,
                    0.3,
                    0.3,
                    0.5,
                    0.5,
                    0.5,
                    0.98
                ],
                "modelname": "ef-d0-v2"
            }
        ]
    }
    return json.dumps(x)

In [2]:
import randomsearch as rs
def your_function(x):
    return -(x[0] ** 2 + x[1] ** 2) + 4   
a, b = rs.optimize(function=your_function, dimensions=2, lower_boundary=[-14, -14], upper_boundary= [10, 10], max_iter=10000, maximize=True)
print(a, ",", b)

3.999314153207934 , [0.01613213083778753, 0.020630102925057514]


# Sample Creater


In [3]:
import time
import shutil 
import pathlib
expname = "Experiment1"
#Cleanup
ClearoldExperimentDATA(expname)
#CopyFiles
productionSRC = r"C:\Users\Amir\Documents\GitHub\mzmses\randomsearch\randomsearch\currentDet"

n_samples = 10
samples = []

trackid = 45
if(len(str(trackid)) == 1):
    trackidSTR = "00"+str(trackid)
elif(len(str(trackid)) == 2):
    trackidSTR = "0"+str(trackid)
else:
    trackidSTR = str(trackid)
src = r"{}\{}".format(productionSRC,trackidSTR)
shutil.copytree(src, expname)

for i in range(n_samples):
    #print(i+1)
    x = MakeSampleRandom(trackid=trackidSTR,precision=1)
    samples.append(x)
    samples_path = r"{}/Configs/{}".format(expname,str(i))
    if not os.path.exists(samples_path):
        os.makedirs(samples_path, exist_ok=True)
    with open(samples_path+"/config.json", "w") as outfile:
        outfile.write(x)

Deleting C:\Users\Amir\Documents\GitHub\mzmses\randomsearch\randomsearch/Experiment1
No Files found, Skipping delete


In [12]:
import numpy as np
X = np.load('res.npy', mmap_mode='r')
len(X)


4

In [39]:
restack = []
for x in range(len(X)):
    resNUmbers = []
    for i in range(8):
        resNUmbers.append(X[x][0][i])
    restack.append(resNUmbers)
restack

[['045', '2022100904504', '255.760', '244.0', '6', '0', '217.0', 'v1L1'],
 ['045', '2022100904503', '113.760', '102.0', '7', '88', '139.6', 'v1L1'],
 ['045', '2022062704504', '157.390', '157.39', '14', '408', '175.0', 'v1L1'],
 ['045', '2021062804501', '101.501', '101.501', '15', '228', '177.2', 'v1L11']]

In [242]:
import glob
import pandas as pd
def ResultsAnalysis(experimentFolder):
    resStack = glob.glob(experimentFolder+r"/**/*.csv")
    for x in resStack[0:1]:
        dfx = pd.DataFrame(x)
        print(x)
def GetConfigScore(dfres):
    Tscore = 0
    for t in dfres['trigger']:
        s = 0
        #MISSS 
        if(t < 12):
            s = 0
            continue

        if(t > 100):
            s = max((300-t)/400,0.25)
            # 50% IS MAX SCORE FOR 
        else:
            s = (113 - t)/100
        #print(t,s)
        Tscore += s
    return Tscore/(len(dfres))

In [249]:

from pathlib import Path
experimentFolder = r"C:\Users\Amir\Documents\GitHub\mzmses\randomsearch\randomsearch\Experiment\Configs"
resStack = glob.glob(experimentFolder+r"/**/*.csv")
scores = np.array(("Config","Score"))
for x in resStack:
    configID = (Path(x).resolve().parent.name)
    dfx = pd.read_csv(x)
    #print(dfx)
    score = GetConfigScore(dfx)
    #print(configID,": ",score)
    a = np.array((configID,score))
    scores = np.vstack((scores,a))
df = pd.DataFrame(scores)
df.columns = df.iloc[0]
df = df[1:]
df['Score'] = df['Score'].astype(float)
df['Score'] = df['Score'].round(2)

print("Top 10")
df[df['Score'] > 0.00].sort_values('Score').head(20)
df.to_csv('ConfigScores.csv',index=False)
df

Top 10


,Config,Score
1,0,0.39
2,1,0.39
3,2,0.39
4,3,0.39
5,4,0.39
6,5,0.39
7,6,0.39
8,7,0.39
9,8,0.39
10,9,0.39
